In [1]:
import simpy
import random
import numpy as np

In [2]:
class Servidor:
    def __init__(self, env, id_servidor):
        self.env = env
        self.id_servidor = id_servidor
        self.procesador = simpy.Resource(env, capacity=1)
        self.carga = 0

    def procesar(self, solicitud):
        # el tiempo de procesamiento del servidor sigue una distribución exponencial
        tiempo_procesamiento = random.expovariate(1 / TIEMPO_PROCESAMIENTO_PROM)
        self.carga += 1
        yield self.env.timeout(tiempo_procesamiento)
        tiempos_atencion_servidores.append(tiempo_procesamiento)
        self.carga -= 1

In [3]:
class BalanceadorRoundRobin:
    def __init__(self, env, servidores):
        self.env = env
        self.servidores = servidores
        self.procesador = simpy.Resource(env, capacity=1)
        self.ultimo_servidor = 0

    def balancear(self, solicitud):
        servidor = None
        with self.procesador.request() as req:
            tiempo_inicio_espera = self.env.now
            yield req
            tiempo_inicio_proceso = self.env.now
            tiempo_de_espera = tiempo_inicio_proceso - tiempo_inicio_espera
            tiempos_espera_cola_balanceador.append(tiempo_de_espera)
            yield self.env.timeout(TIEMPO_BALANCEADOR)
            servidor = self.seleccionar_servidor()
        
        with servidor.procesador.request() as req:
            tiempo_inicio_espera = self.env.now
            yield req
            tiempos_espera_cola_servidores.append(self.env.now - tiempo_inicio_espera)
            yield self.env.process(servidor.procesar(solicitud))

    def seleccionar_servidor(self):
        servidor = self.servidores[self.ultimo_servidor]
        self.ultimo_servidor = (self.ultimo_servidor + 1) % len(self.servidores)
        return servidor


In [4]:
class BalanceadorDifuso:
    def __init__(self, env, servidores, grafo, umbral):
        self.env = env
        self.servidores = servidores
        self.procesador = simpy.Resource(env, capacity=1)
        self.grafo = grafo
        self.umbral = umbral

    def balancear(self, solicitud):
        servidor = None
        with self.procesador.request() as req:
            tiempo_inicio_espera = self.env.now
            yield req
            tiempos_espera_cola_balanceador.append(self.env.now - tiempo_inicio_espera)
            yield self.env.timeout(TIEMPO_BALANCEADOR)
            servidor = self.seleccionar_servidor()

        with servidor.procesador.request() as req:
            tiempo_inicio_espera = self.env.now
            yield req
            tiempos_espera_cola_servidores.append(self.env.now - tiempo_inicio_espera)
            yield self.env.process(servidor.procesar(solicitud))

    def seleccionar_servidor(self):
        # selecciona un servidor con menor carga siguiendo el grafo y el umbral de carga
        servidor_inicial = random.choice(self.servidores) # aca por ahi podriamos hacer un round robin como dice en el paper
        return self.propagacion_difusa(servidor_inicial)

    def propagacion_difusa(self, servidor_actual):
        # busca el servidor menos cargado entre los vecinos si la diferencia de carga excede el umbral
        visitados = set()  # para evitar ciclos en el grafo
        cola = [servidor_actual]

        while cola:
            servidor = cola.pop(0)
            visitados.add(servidor.id_servidor)

            vecinos = [self.servidores[vid] for vid in self.grafo[servidor.id_servidor] if vid not in visitados]
            if not vecinos:
                break  # no hay vecinos a los que propagar

            servidor_menos_cargado = min(vecinos, key=lambda s: s.carga)
            diferencial_carga = servidor.carga - servidor_menos_cargado.carga

            if diferencial_carga > self.umbral:
                cola.append(servidor_menos_cargado)  # propaga al vecino menos cargado
            else:
                return servidor  # retorna el servidor actual si la carga es aceptable

        return servidor_actual  # devuelve el servidor inicial si no se encontró uno mejor

In [5]:
def generar_solicitudes(env, balanceador):
    solicitud_id = 0

    for _ in range(CANTIDAD_SOLICITUDES):
        env.process(balanceador.balancear(solicitud_id))
        solicitud_id += 1
        yield env.timeout(random.expovariate(TASA_SOLICITUDES))

In [6]:
def simular(tecnica, num_servidores):
    env = simpy.Environment()
    servidores = [Servidor(env, i) for i in range(num_servidores)]
    
    if tecnica == "Difuso":
        balanceador = BalanceadorDifuso(env, servidores, grafo, UMBRAL_PROPAGACION)
    elif tecnica == "Round Robin":
        balanceador = BalanceadorRoundRobin(env, servidores)
    else:
        raise ValueError("Técnica no soportada en esta simulación")

    env.process(generar_solicitudes(env, balanceador))
    env.run()

In [7]:
def mostrar_resultados():
    print("Media de espera en cola de balanceador:", np.mean(tiempos_espera_cola_balanceador))
    print("Media de espera en cola de servidores:", np.mean(tiempos_espera_cola_servidores))
    print("Media de atención en servidores:", np.mean(tiempos_atencion_servidores))
    print()

    print("Mediana de espera en cola de balanceador:", np.median(tiempos_espera_cola_balanceador))
    print("Mediana de espera en cola de servidores:", np.median(tiempos_espera_cola_servidores))
    print("Mediana de atención en servidores:", np.median(tiempos_atencion_servidores))
    print()
    
    print("Desviación estándar de espera en cola del balanceador:", np.std(tiempos_espera_cola_balanceador))
    print("Desviación estándar de espera en cola de servidores:", np.std(tiempos_espera_cola_servidores))
    print("Desviación estándar de atención en servidores:", np.std(tiempos_atencion_servidores))

In [8]:
# Parámetros de la simulación
TASA_SOLICITUDES = 5                # Tasa de llegada de solicitudes en solicitudes/segundo (poisson)
TIEMPO_BALANCEADOR = 0.01           # Tiempo que toma al balanceador derivar la solicitud en segundos
TIEMPO_PROCESAMIENTO_PROM = 0.1     # Tiempo promedio de procesamiento en segundos
NUM_SERVIDORES = 5                  # Número de servidores

CANTIDAD_SOLICITUDES = 100_000      # Cantidad de solicitudes a ejecutar en la simulacion

# Inicialización de métricas
tiempos_espera_cola_balanceador = [] # Tiempos de espera entre la llegada de una solicitud al balanceador y la atencion del mismo
tiempos_espera_cola_servidores = []  # Tiempos de espera entre la llegada de una solicitud al servidor y la atencion del mismo
tiempos_atencion_servidores = []     # Tiempos de procesamiento de las solicitudes en el servidor

print("Simulación Round Robin\n")
simular("Round Robin", NUM_SERVIDORES)
mostrar_resultados()

Simulación Round Robin

Media de espera en cola de balanceador: 0.0002593490414312376
Media de espera en cola de servidores: 0.00041339444089973993
Media de atención en servidores: 0.10024531968572888

Mediana de espera en cola de balanceador: 0.0
Mediana de espera en cola de servidores: 0.0
Mediana de atención en servidores: 0.06968501726776714

Desviación estándar de espera en cola del balanceador: 0.0013330137141183327
Desviación estándar de espera en cola de servidores: 0.008984400375841072
Desviación estándar de atención en servidores: 0.10002393737172793


In [9]:
# Parámetros de la simulación
TASA_SOLICITUDES = 5                # Tasa de llegada de solicitudes en solicitudes/segundo (poisson)
TIEMPO_BALANCEADOR = 0.01           # Tiempo que toma al balanceador derivar la solicitud en segundos
TIEMPO_PROCESAMIENTO_PROM = 0.1     # Tiempo promedio de procesamiento en segundos
NUM_SERVIDORES = 5                  # Número de servidores

UMBRAL_PROPAGACION = 2              # Diferencia de carga para propagación en balanceador difuso
CANTIDAD_SOLICITUDES = 100_000      # Cantidad de solicitudes a ejecutar en la simulacion

# Definición de la red de servidores en forma de grafo (lista de adyacencia)
grafo = {
    0: [1, 2],
    1: [0, 3, 4],
    2: [0, 3],
    3: [1, 2, 4],
    4: [1, 3]
}

# Inicialización de métricas
tiempos_espera_cola_balanceador = [] # Tiempos que tarda el balanceador en enviar las solicitudes al servidor
tiempos_espera_cola_servidores = []  # Tiempos de espera entre la llegada de una solicitud al servidor y la atencion del mismo
tiempos_atencion_servidores = []     # Tiempos de procesamiento de las solicitudes en el servidor

print("Simulación Difusión\n")
simular("Difuso", NUM_SERVIDORES)
mostrar_resultados()

Simulación Difusión

Media de espera en cola de balanceador: 0.0002670355659791144
Media de espera en cola de servidores: 0.011000204795488086
Media de atención en servidores: 0.09995366530266565

Mediana de espera en cola de balanceador: 0.0
Mediana de espera en cola de servidores: 0.0
Mediana de atención en servidores: 0.0689940288001283

Desviación estándar de espera en cola del balanceador: 0.001354000985562318
Desviación estándar de espera en cola de servidores: 0.0478443698652636
Desviación estándar de atención en servidores: 0.10057835547748946
